In [ ]:
!pip install -q streamlit pandas opencv-python-headless Pillow tensorflow "scikit-learn>=1.0" pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 86.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.7 MB/s eta 0:00:00:00:0100:01


In [ ]:
%%writefile app.py

import streamlit as st
import os
import pandas as pd
import numpy as np
import pickle
from PIL import Image
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
import random
import cv2

# --- Page Configuration ---
st.set_page_config(layout="wide", page_title="AI Fashion Stylist")

# --- PATHS ---
# IMPORTANT: Update these folder names to match your Kaggle datasets
SAVED_OUTPUT_DIR = "/kaggle/input/final-recommendation" 
BASE_DATA_DIR = "/kaggle/input/clothestry/clothes_tryon_dataset"
STYLE_CSV_PATH = "/kaggle/input/description-and-style/cloth_style_profile.csv" 
DESC_CSV_PATH = "/kaggle/input/description-and-style/cloth_description_profile.csv"
FEATURES_PATH = "/kaggle/input/embeddings/embeddings_full_dataset.pkl"
MAP_PATH = os.path.join(SAVED_OUTPUT_DIR, "file_split_map.pkl") if os.path.exists(SAVED_OUTPUT_DIR) else "" # Handle if map is missing
UPLOAD_DIR = "/kaggle/working/uploads"

if not os.path.exists(UPLOAD_DIR): os.makedirs(UPLOAD_DIR)

# --- Caching Functions ---
@st.cache_data
def load_data_and_embeddings():
    style_df = pd.read_csv(STYLE_CSV_PATH)
    desc_df = pd.read_csv(DESC_CSV_PATH)
    df = pd.merge(style_df, desc_df, on='filename', how='inner')
    
    with open(FEATURES_PATH, 'rb') as f: embeddings_list = pickle.load(f)
    
    # Re-create the file_map_df on the fly as a robust fallback
    train_files = sorted([f for f in os.listdir(os.path.join(BASE_DATA_DIR, "train/cloth")) if f.endswith('.jpg')])
    test_files = sorted([f for f in os.listdir(os.path.join(BASE_DATA_DIR, "test/cloth")) if f.endswith('.jpg')])
    all_files_in_order = train_files + test_files
    
    file_map_df = pd.DataFrame({'filename': all_files_in_order, 'split': ['train'] * len(train_files) + ['test'] * len(test_files)})
    
    min_len = min(len(embeddings_list), len(file_map_df))
    file_map_df = file_map_df.iloc[:min_len]
    embeddings_list = embeddings_list[:min_len]

    file_map_df['resnet_embedding'] = embeddings_list
    final_df = pd.merge(df, file_map_df, on='filename', how='inner')
    
    final_df['cloth_path'] = final_df.apply(lambda r: os.path.join(BASE_DATA_DIR, r['split'], 'cloth', r['filename']), axis=1)
    final_df['model_path'] = final_df.apply(lambda r: os.path.join(BASE_DATA_DIR, r['split'], 'image', r['filename']), axis=1)
    final_df['search_text'] = (final_df['style_category'] + ' ' + final_df['description']).str.lower()
    
    # Get dominant color RGB for color harmony
    final_df['rgb_values'] = final_df['dominant_color_rgb'].apply(lambda x: tuple(map(int, x.strip("()").split(','))) if isinstance(x, str) else (0,0,0))

    return final_df

@st.cache_resource
def get_recommendation_model(_df):
    embeddings = np.array(_df['resnet_embedding'].tolist())
    neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='cosine').fit(embeddings)
    return neighbors

# --- NEW: Personalized Recommendation Logic ---

@st.cache_resource
def load_face_hair_cascade():
    # Using pre-trained Haar Cascades for detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    return face_cascade

def get_dominant_color(image, k=3):
    image = image.reshape((image.shape[0] * image.shape[1], 3))
    clt = KMeans(n_clusters=k, n_init='auto')
    clt.fit(image)
    # Return the most dominant color cluster center
    return clt.cluster_centers_[np.argmax(np.unique(clt.labels_, return_counts=True)[1])]

def analyze_facial_features(image_path, face_cascade):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bgr_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # For display in Streamlit
    
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    if len(faces) == 0:
        return None, None, None

    # Use the largest detected face
    (x, y, w, h) = sorted(faces, key=lambda f: f[2]*f[3], reverse=True)[0]
    
    
    # Crop to the center of the face (cheeks/forehead) to avoid beards/lips
    face_roi = img[y + int(h*0.2):y + int(h*0.6), x + int(w*0.2):x + int(w*0.8)]
    dominant_skin_color = get_dominant_color(face_roi)

    # Approximate hair region as the area above the face
    hair_y_end = y + int(h*0.3)
    hair_roi = img[max(0, y-int(h*0.4)):hair_y_end, x:x+w]
    if hair_roi.shape[0] == 0 or hair_roi.shape[1] == 0:
        dominant_hair_color = (0,0,0) # Default if no hair is found
    else:
        dominant_hair_color = get_dominant_color(hair_roi)
    
    # Draw rectangles for visualization
    cv2.rectangle(bgr_img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    cv2.rectangle(bgr_img, (x, max(0, y-int(h*0.4))), (x+w, hair_y_end), (0, 255, 0), 2) # Hair region
    
    return dominant_skin_color, dominant_hair_color, bgr_img

def classify_tone_and_color(rgb_skin, rgb_hair):
    # Simplified classification based on brightness (Luminance)
    skin_brightness = 0.299*rgb_skin[0] + 0.587*rgb_skin[1] + 0.114*rgb_skin[2]
    if skin_brightness > 180: skin_tone = "Light"
    elif skin_brightness > 110: skin_tone = "Medium"
    else: skin_tone = "Dark"
    
    # Simplified classification for hair color
    r, g, b = rgb_hair
    if r > 100 and g > 100 and b < 100: hair_color = "Blonde"
    elif r > 120 and g < 100 and b < 100: hair_color = "Red"
    elif max(r,g,b) < 80: hair_color = "Black"
    else: hair_color = "Brown"
        
    return skin_tone, hair_color

def get_color_harmony_recommendations(skin_tone, hair_color, df, num_recs=10):
    # --- Color Harmony Rules (This can be greatly expanded) ---
    harmony_rules = {
        "Light": [(10, 60, 100), (200, 150, 150)], # Blues, Pastels
        "Medium": [(0, 100, 0), (128, 0, 128)], # Greens, Purples
        "Dark": [(255, 0, 0), (0, 0, 255)], # Vibrant Reds, Blues
    }
    target_colors = harmony_rules.get(skin_tone, harmony_rules["Medium"])
    
    all_cloth_colors = np.array(df['rgb_values'].tolist())
    
    # Find clothes closest to our target harmony colors
    scores = []
    for target_color in target_colors:
        distances = np.sqrt(np.sum((all_cloth_colors - target_color)**2, axis=1))
        scores.append(distances)
        
    # Average the distances and find the indices with the smallest average distance
    avg_scores = np.mean(scores, axis=0)
    recommended_indices = np.argsort(avg_scores)[:num_recs]
    
    return df.iloc[recommended_indices]


# Load data and models
df = load_data_and_embeddings()
resnet_neighbors = get_recommendation_model(df)
face_cascade = load_face_hair_cascade()

st.title("👗 AI Personal Fashion Stylist")

# Create tabs for different functionalities
tab1, tab2 = st.tabs(["Personalized Recommendations", "Catalog & Visual Search"])

with tab1:
    st.header("Upload Your Photo for Personalized Styling")
    st.write("Our AI will analyze your skin tone and hair color to recommend outfits that complement your features.")
    
    uploaded_file = st.file_uploader("Choose your image...", type=['jpg', 'jpeg', 'png'], key="persona_upload")
    
    if uploaded_file:
        file_path = os.path.join(UPLOAD_DIR, uploaded_file.name)
        with open(file_path, "wb") as f: f.write(uploaded_file.getbuffer())
        
        with st.spinner("Analyzing your features..."):
            skin_rgb, hair_rgb, annotated_img = analyze_facial_features(file_path, face_cascade)
        
        if skin_rgb is not None:
            skin_tone, hair_color = classify_tone_and_color(skin_rgb, hair_rgb)
            
            col1, col2 = st.columns(2)
            with col1:
                st.image(annotated_img, channels="BGR", caption="AI Analysis (Face in Blue, Hair in Green)")
            with col2:
                st.subheader("Your AI Style Profile:")
                st.write(f"**Detected Skin Tone:** {skin_tone}")
                st.write(f"**Detected Hair Color:** {hair_color}")
                st.color_picker("Dominant Skin Tone", value=f"#{int(skin_rgb[0]):02x}{int(skin_rgb[1]):02x}{int(skin_rgb[2]):02x}")
                st.color_picker("Dominant Hair Color", value=f"#{int(hair_rgb[0]):02x}{int(hair_rgb[1]):02x}{int(hair_rgb[2]):02x}")

            st.markdown("---")
            st.subheader(f"🎨 Recommended Colors for You")
            
            rec_df = get_color_harmony_recommendations(skin_tone, hair_color, df)
            st.image([c for c in rec_df['cloth_path']], width=120, caption=[s for s in rec_df['style_category']])
            
        else:
            st.error("Could not detect a face in the uploaded image. Please try a clearer, front-facing photo.")

with tab2:
    # This is your previous app logic, now neatly in a tab
    st.header("Catalog Search & Visual Recommendations")
    if 'view' not in st.session_state: st.session_state.view = 'catalog'
    if 'selected_item_idx' not in st.session_state: st.session_state.selected_item_idx = None
    
    if st.session_state.view == 'detail':
        item = df.iloc[st.session_state.selected_item_idx]
        if st.button("⬅️ Back to Catalog", key="back_btn"): st.session_state.view = 'catalog'; st.rerun()
        # Detail view layout here...
        st.header(f"Details for {item['style_category']}")
        col1, col2 = st.columns(2)
        with col1: st.image(item['model_path'], caption="Model View", use_container_width=True)
        with col2: st.image(item['cloth_path'], caption="Garment View", use_container_width=True)
        st.subheader(f"**AI Description:** *{item['description']}*")
        st.markdown("---"); st.header("✨ Visually Similar Items (ResNet)")
        item_embedding = item['resnet_embedding'].reshape(1, -1)
        _, indices = resnet_neighbors.kneighbors(item_embedding)
        rec_df = df.iloc[indices[0][1:]]
        st.image([c for c in rec_df['cloth_path']], width=120, caption=[s for s in rec_df['style_category']])
    else:
        # Catalog view layout here...
        st.sidebar.header("🔍 Search & Filter")
        search_query = st.sidebar.text_input("Search by description")
        style_options = ['All'] + sorted(df['style_category'].unique().tolist())
        sel_style = st.sidebar.selectbox("Filter by Style Category:", style_options)
        
        filtered_df = df.copy()
        if search_query:
            for term in search_query.lower().split(): filtered_df = filtered_df[filtered_df['search_text'].str.contains(term, na=False)]
        if sel_style != 'All': filtered_df = filtered_df[filtered_df['style_category'] == sel_style]

        st.header(f"Browse Our Collection ({len(filtered_df)} items found)")
        items_per_page = 20; page_num = st.sidebar.number_input(f'Page', 1, max(1, (len(filtered_df)-1)//items_per_page+1), 1)
        start, end = (page_num - 1) * items_per_page, page_num * items_per_page
        paginated_df = filtered_df.iloc[start:end]
        
        cols = st.columns(5)
        for i, (idx, row) in enumerate(paginated_df.iterrows()):
            with cols[i % 5]:
                st.image(row['cloth_path'], use_container_width=True)
                st.caption(row['style_category'])
                if st.button("View Details", key=f"view_{idx}"):
                    st.session_state.selected_item_idx = idx; st.session_state.view = 'detail'; st.rerun()

Writing app.py


In [ ]:

from kaggle_secrets import UserSecretsClient
from pyngrok import ngrok
import os
import time

# Get your ngrok token from secrets
user_secrets = UserSecretsClient()
NGROK_AUTH_TOKEN = user_secrets.get_secret("NGROK_AUTH_TOKEN")

# Authenticate and create tunnel
os.system("killall ngrok")
time.sleep(2)
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(8501)

# Print the public URL for you to click
print("="*65)
print(f" Your Streamlit App is LIVE! Click the link below to open it: 🎉")
print(public_url)
print("="*65)

# Run the Streamlit app
!streamlit run app.py

ngrok: no process found


🎉 Your Streamlit App is LIVE! Click the link below to open it: 🎉
NgrokTunnel: "https://f5f0c6a4bdd2.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://34.90.130.73:8501

